##### Bibliotecas

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import nasdaqdatalink
import numpy as np
import numpy_financial as npf
import datetime

In [3]:
nasdaq_api_key = 'yUkkodwj1uqL1EPc1kdS'
nasdaqdatalink.ApiConfig.api_key = nasdaq_api_key

##### Início da coleta de dados

In [4]:
start_date = '31/01/2000'
end_date = '31/12/2023'

In [5]:
selic = pd.DataFrame()
ipca = pd.DataFrame()

In [6]:
selic['SELIC'] = nasdaqdatalink.get('BCB/432', start_date=start_date,
                                    end_date=end_date,
                                    collapse='monthly')
ipca['IPCA'] = nasdaqdatalink.get('BCB/13522', start_date=start_date,
                                  end_date=end_date,
                                  collapes='monthly')

In [140]:
selic

,SELIC
Date,
2000-01-31,19.00
2000-02-29,19.00
2000-03-31,18.50
2000-04-30,18.50
2000-05-31,18.50
...,...
2023-05-31,13.75
2023-06-30,13.75
2023-07-31,13.75


In [141]:
ipca

,IPCA
Date,
2000-01-31,8.85
2000-02-29,7.86
2000-03-31,6.92
2000-04-30,6.77
2000-05-31,6.47
...,...
2023-04-30,4.18
2023-05-31,3.94
2023-06-30,3.16


In [7]:
data = pd.concat([selic, ipca], axis=1)
data

,SELIC,IPCA
Date,,
2000-01-31,19.00,8.85
2000-02-29,19.00,7.86
2000-03-31,18.50,6.92
2000-04-30,18.50,6.77
2000-05-31,18.50,6.47
...,...,...
2023-05-31,13.75,3.94
2023-06-30,13.75,3.16
2023-07-31,13.75,3.99


##### Verificação do Spread entre Selic e IPCA

In [8]:
data['SPREAD'] = data['SELIC'] - data['IPCA']
data

,SELIC,IPCA,SPREAD
Date,,,
2000-01-31,19.00,8.85,10.15
2000-02-29,19.00,7.86,11.14
2000-03-31,18.50,6.92,11.58
2000-04-30,18.50,6.77,11.73
2000-05-31,18.50,6.47,12.03
...,...,...,...
2023-05-31,13.75,3.94,9.81
2023-06-30,13.75,3.16,10.59
2023-07-31,13.75,3.99,9.76


In [144]:
media_spread = round(np.mean(data['SPREAD']), 2)
media_spread

5.96

##### Plotagem do gráfico com Selic, IPCA e Spread

In [145]:
fig = make_subplots(rows=1, cols=1, specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(name='SELIC', x=data.index, 
                         y=data['SELIC']), row=1, col=1)
fig.add_trace(go.Scatter(name='IPCA', x=data.index, 
                         y=data['IPCA']), row=1, col=1, 
              secondary_y=False)
fig.add_trace(go.Scatter(name='SPREAD', x=data.index, 
                         y=data['SPREAD']), row=1, col=1, 
              secondary_y=False)

fig.update_layout(title_text= 'SPREAD DE SELIC vs IPCA', 
                  template='simple_white', width=1000, height=800)
fig.show()

##### Criação do gráfico de range

In [146]:
sd1_min_spread = media_spread - round(np.std(data['SPREAD']), 2)
sd1_max_spread = media_spread + round(np.std(data['SPREAD']), 2)
sd2_min_spread = media_spread - (2 * (round(np.std(data['SPREAD']), 2)))
sd2_max_spread = media_spread + (2 * (round(np.std(data['SPREAD']), 2)))

In [147]:
print(sd1_min_spread)
print(sd1_max_spread)
print(sd2_min_spread)
print(sd2_max_spread)

1.9000000000000004
10.02
-2.1599999999999993
14.079999999999998


In [148]:
fig = px.line(data, x=data.index, y=data['SPREAD'])

fig.add_hline(y=media_spread, line_width=5, line_color='blue')
fig.add_hline(y=0, line_width=5, line_color='black')
fig.add_hline(y=sd1_min_spread, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=sd1_max_spread, line_width=3, line_dash='dash', line_color='green')
fig.add_hline(y=sd2_min_spread, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=sd2_max_spread, line_width=3, line_dash='dash', line_color='green')

fig.update_layout(title_text='Spread SELIC vs IPCA',
                  template='simple_white', width=1000, height=800)
fig.show()

##### Simulação de investimento

In [317]:
df = data.copy(True)

In [318]:
df

,SELIC,IPCA,SPREAD
Date,,,
2000-01-31,19.00,8.85,10.15
2000-02-29,19.00,7.86,11.14
2000-03-31,18.50,6.92,11.58
2000-04-30,18.50,6.77,11.73
2000-05-31,18.50,6.47,12.03
...,...,...,...
2023-05-31,13.75,3.94,9.81
2023-06-30,13.75,3.16,10.59
2023-07-31,13.75,3.99,9.76


In [158]:
pd.options.display.float_format = '{:.2f}'.format

In [319]:
aporte_inicial = 10000

In [320]:
taxas = {'SELIC 100%': (1 + df['SELIC'] / 100) ** (1 / 12) - 1,
         'SELIC 110%': ((1 + df['SELIC'] / 100) ** (1 / 12) - 1) * 1.1,
         'IPCA + 4%': (1 + df['IPCA'] / 100 + 0.04) ** (1 / 12) - 1,
         'IPCA + 5%': (1 + df['IPCA'] / 100 + 0.05) ** (1 / 12) - 1,
         'IPCA + 6%': (1 + df['IPCA'] / 100 + 0.06) ** (1 / 12) - 1,
         'IPCA + 7%': (1 + df['IPCA'] / 100 + 0.07) ** (1 / 12) - 1
}

In [321]:
for taxa_nome, taxa in taxas.items():
    aporte_inicial_temp = aporte_inicial
    patrimonio = [aporte_inicial_temp]
    for i in range(1, len(df)):
        aporte_inicial_temp = aporte_inicial_temp * (1 + taxa.iloc[i])
        patrimonio.append(aporte_inicial_temp)
    df[taxa_nome] = patrimonio

In [322]:
df

,SELIC,IPCA,SPREAD,SELIC 100%,SELIC 110%,IPCA + 4%,IPCA + 5%,IPCA + 6%,IPCA + 7%
Date,,,,,,,,,
2000-01-31,19.00,8.85,10.15,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
2000-02-29,19.00,7.86,11.14,10146.02,10160.62,10093.84,10101.32,10108.75,10116.12
2000-03-31,18.50,6.92,11.58,10290.55,10319.84,10181.39,10196.57,10211.63,10226.59
2000-04-30,18.50,6.77,11.73,10437.15,10481.55,10268.54,10291.56,10314.42,10337.12
2000-05-31,18.50,6.47,12.03,10585.84,10645.80,10354.11,10385.11,10415.92,10446.56
...,...,...,...,...,...,...,...,...,...
2023-05-31,13.75,3.94,9.81,146301.67,191015.25,99899.07,123308.28,151916.51,186816.04
2023-06-30,13.75,3.16,10.59,147880.84,193283.23,100476.43,124116.96,153030.12,188328.55
2023-07-31,13.75,3.99,9.76,149477.05,195578.14,101122.12,125010.55,154249.24,189972.10


##### Gráfico de patrimônio

In [323]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(name='SELIC 100%', x=df.index, 
                         y=df['SELIC 100%']), row=1, col=1)
fig.add_trace(go.Scatter(name='SELIC 110%', x=df.index, 
                         y=df['SELIC 110%']), row=1, col=1)
fig.add_trace(go.Scatter(name='IPCA + 4%', x=df.index, 
                         y=df['IPCA + 4%']), row=1, col=1)
fig.add_trace(go.Scatter(name='IPCA + 5%', x=df.index, 
                         y=df['IPCA + 5%']), row=1, col=1)
fig.add_trace(go.Scatter(name='IPCA + 6%', x=df.index, 
                         y=df['IPCA + 6%']), row=1, col=1)
fig.add_trace(go.Scatter(name='IPCA + 7%', x=df.index, 
                         y=df['IPCA + 7%']), row=1, col=1)

fig.update_layout(title_text= 'Evolução do Patrimônio Investindo R$ 10.000', 
                  template='simple_white', width=1000, height=800)
fig.show()